### Imports

In [1]:
from tqdm import tqdm
from utils.utils import *

import json
import networkx as nx
import numpy as np
import pandas as pd
import os
import sys

%load_ext autotime

/home/ealvarez/miniconda3/envs/Conda_dgl_3.10/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/ealvarez/miniconda3/envs/Conda_dgl_3.10/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/ealvarez/miniconda3/envs/Conda_dgl_3.10/li

time: 181 µs (started: 2023-06-04 19:02:18 -05:00)


### Parameters

In [2]:
dir_path = "output"

res = sorted(os.listdir(dir_path))
n = len(res) + 1
exp = "exp{}".format(n)
exp

'exp1'

time: 5.39 ms (started: 2023-06-04 19:02:18 -05:00)


In [3]:
EXP = exp
RAW_DATA_FOLDER = "Edwin_proyecto2" # change

time: 649 µs (started: 2023-06-04 19:02:18 -05:00)


### Load dataset

In [4]:
# load dataset groups
df1 = pd.read_csv("temp/{}/{}.csv".format(RAW_DATA_FOLDER, "int1"), delimiter="|")
df2 = pd.read_csv("temp/{}/{}.csv".format(RAW_DATA_FOLDER, "int2"), delimiter="|")
df3 = pd.read_csv("temp/{}/{}.csv".format(RAW_DATA_FOLDER, "int3"), delimiter="|")
df4 = pd.read_csv("temp/{}/{}.csv".format(RAW_DATA_FOLDER, "int4"), delimiter="|")
df5 = pd.read_csv("temp/{}/{}.csv".format(RAW_DATA_FOLDER, "int5"), delimiter="|")

time: 278 ms (started: 2023-06-04 19:02:18 -05:00)


In [5]:
df1

,ionIdx,ionMz,0001 / zwf1^ 3.4,0002 / zwf1^ 3.4,0003 / zwf1^ 3.4,0004 / zwf1^ 3.4,0005 / zwf1^ 3.4,0006 / zwf1^ 3.4,0007 / zwf1^ 3.4,0008 / zwf1^ 3.4,...,0161 / zwf1^ 3.2,0162 / zwf1^ 3.2,0163 / zwf1^ 3.2,0164 / zwf1^ 3.2,0165 / zwf1^ 3.2,0166 / zwf1^ 3.2,0167 / zwf1^ 3.2,0168 / zwf1^ 3.2,0169 / zwf1^ 3.2,0170 / zwf1^ 3.2
0,1,59.0049,1,47,508,360,675,232,59,345,...,1,220,236,662,875,502,289,603,848,312
1,2,59.0137,53182,57902,51861,62812,54291,54395,59054,52547,...,87775,74295,76910,40959,80929,84501,92848,68262,79165,61671
2,3,59.0291,2151,2102,2334,2864,2393,2246,2331,1954,...,2989,3407,2992,2282,3544,3571,3768,2761,3339,2605
3,4,59.0370,83,174,366,1134,693,464,235,479,...,335,898,327,948,893,913,1089,630,430,516
4,5,59.0453,1,1,51,642,493,143,11,227,...,93,129,236,495,694,167,589,147,5,51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6240,6241,996.5509,2470,2435,2465,2727,2773,2157,2168,2623,...,2912,2626,2664,1151,2495,2031,3171,2693,2054,2115
6241,6242,996.7096,3366,3241,2909,3469,3538,3010,3031,2872,...,3124,2707,3320,1508,2814,3212,3853,3465,2944,2625
6242,6243,997.5542,2818,2335,1807,1679,2222,2070,2158,1835,...,2677,2176,2154,2074,1460,2206,1661,2267,2025,1719
6243,6244,997.7131,1901,1469,1572,1916,2180,2555,2101,1656,...,2121,1831,1981,960,1346,1623,1721,1549,1703,2259


time: 23.1 ms (started: 2023-06-04 19:02:18 -05:00)


### Format dataset

In [6]:
# concat
df_join_raw = pd.concat([
    df1.iloc[:, 1:], 
    df2.iloc[:, 2:], 
    df3.iloc[:, 2:], 
    df4.iloc[:, 2:]], axis=1)
df_join_raw.set_index("ionMz", inplace=True)

print(df_join_raw.shape)
df_join_raw = df_join_raw.rename_axis(None)
df_join_raw

(6245, 400)


,0001 / zwf1^ 3.4,0002 / zwf1^ 3.4,0003 / zwf1^ 3.4,0004 / zwf1^ 3.4,0005 / zwf1^ 3.4,0006 / zwf1^ 3.4,0007 / zwf1^ 3.4,0008 / zwf1^ 3.4,0009 / zwf1^ 3.4,0010 / zwf1^ 3.4,...,0951 / WT 3.4,0952 / WT 3.4,0953 / WT 3.4,0954 / WT 3.4,0955 / WT 3.4,0956 / WT 3.4,0957 / WT 3.4,0958 / WT 3.4,0959 / WT 3.4,0960 / WT 3.4
59.0049,1,47,508,360,675,232,59,345,346,504,...,45,817,32,307,410,716,228,187,361,952
59.0137,53182,57902,51861,62812,54291,54395,59054,52547,57544,63538,...,45747,39677,47205,38697,54320,62610,47283,49927,43632,48511
59.0291,2151,2102,2334,2864,2393,2246,2331,1954,2310,2528,...,1852,1717,1830,1660,2006,2539,2103,2276,1643,2522
59.0370,83,174,366,1134,693,464,235,479,443,691,...,225,349,185,429,186,278,364,368,115,887
59.0453,1,1,51,642,493,143,11,227,160,154,...,60,293,1,336,65,142,1,182,10,684
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996.5509,2470,2435,2465,2727,2773,2157,2168,2623,2256,2636,...,1866,1834,1669,674,2533,1017,2983,1814,2160,1899
996.7096,3366,3241,2909,3469,3538,3010,3031,2872,2709,3601,...,1748,1419,1160,621,1758,1474,1973,1023,1769,1629
997.5542,2818,2335,1807,1679,2222,2070,2158,1835,2087,2784,...,1168,828,1162,1103,2253,1419,2306,1542,827,1433
997.7131,1901,1469,1572,1916,2180,2555,2101,1656,2543,2643,...,659,1370,714,390,1660,1513,1843,1448,691,1376


time: 43.6 ms (started: 2023-06-04 19:02:18 -05:00)


In [7]:
# get groups name
print(df_join_raw.columns.values)
groups_id = ["WT", "zwf1^", "pck1^"]

['0001 / zwf1^ 3.4' '0002 / zwf1^ 3.4' '0003 / zwf1^ 3.4'
 '0004 / zwf1^ 3.4' '0005 / zwf1^ 3.4' '0006 / zwf1^ 3.4'
 '0007 / zwf1^ 3.4' '0008 / zwf1^ 3.4' '0009 / zwf1^ 3.4'
 '0010 / zwf1^ 3.4' '0011 / pck1^ 2.4' '0012 / pck1^ 2.4'
 '0013 / pck1^ 2.4' '0014 / pck1^ 2.4' '0015 / pck1^ 2.4'
 '0016 / pck1^ 2.4' '0017 / pck1^ 2.4' '0018 / pck1^ 2.4'
 '0019 / pck1^ 2.4' '0020 / pck1^ 2.4' '0021 / zwf1^ 1.2'
 '0022 / zwf1^ 1.2' '0023 / zwf1^ 1.2' '0024 / zwf1^ 1.2'
 '0025 / zwf1^ 1.2' '0026 / zwf1^ 1.2' '0027 / zwf1^ 1.2'
 '0028 / zwf1^ 1.2' '0029 / zwf1^ 1.2' '0030 / zwf1^ 1.2'
 '0031 / pck1^ 1.2' '0032 / pck1^ 1.2' '0033 / pck1^ 1.2'
 '0034 / pck1^ 1.2' '0035 / pck1^ 1.2' '0036 / pck1^ 1.2'
 '0037 / pck1^ 1.2' '0038 / pck1^ 1.2' '0039 / pck1^ 1.2'
 '0040 / pck1^ 1.2' '0061 / pck1^ 1.3' '0062 / pck1^ 1.3'
 '0063 / pck1^ 1.3' '0064 / pck1^ 1.3' '0065 / pck1^ 1.3'
 '0066 / pck1^ 1.3' '0067 / pck1^ 1.3' '0068 / pck1^ 1.3'
 '0069 / pck1^ 1.3' '0070 / pck1^ 1.3' '0071 / WT 2.2' '0072 / WT 2.2'
 

In [8]:
# change columns name (AB1.1)

columns = df_join_raw.columns.values

for i in range(len(columns)):
    # remove white space
    columns[i] = columns[i].replace(" ", "")
    columns[i] = columns[i].split("/")[1] 

df_join_raw.columns = columns
df_join_raw

,zwf1^3.4,zwf1^3.4,zwf1^3.4,zwf1^3.4,zwf1^3.4,zwf1^3.4,zwf1^3.4,zwf1^3.4,zwf1^3.4,zwf1^3.4,...,WT3.4,WT3.4,WT3.4,WT3.4,WT3.4,WT3.4,WT3.4,WT3.4,WT3.4,WT3.4
59.0049,1,47,508,360,675,232,59,345,346,504,...,45,817,32,307,410,716,228,187,361,952
59.0137,53182,57902,51861,62812,54291,54395,59054,52547,57544,63538,...,45747,39677,47205,38697,54320,62610,47283,49927,43632,48511
59.0291,2151,2102,2334,2864,2393,2246,2331,1954,2310,2528,...,1852,1717,1830,1660,2006,2539,2103,2276,1643,2522
59.0370,83,174,366,1134,693,464,235,479,443,691,...,225,349,185,429,186,278,364,368,115,887
59.0453,1,1,51,642,493,143,11,227,160,154,...,60,293,1,336,65,142,1,182,10,684
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996.5509,2470,2435,2465,2727,2773,2157,2168,2623,2256,2636,...,1866,1834,1669,674,2533,1017,2983,1814,2160,1899
996.7096,3366,3241,2909,3469,3538,3010,3031,2872,2709,3601,...,1748,1419,1160,621,1758,1474,1973,1023,1769,1629
997.5542,2818,2335,1807,1679,2222,2070,2158,1835,2087,2784,...,1168,828,1162,1103,2253,1419,2306,1542,827,1433
997.7131,1901,1469,1572,1916,2180,2555,2101,1656,2543,2643,...,659,1370,714,390,1660,1513,1843,1448,691,1376


time: 24.4 ms (started: 2023-06-04 19:02:19 -05:00)


In [9]:
# get subgroups names
subgroups_id = get_subgroups_id(df_join_raw, groups_id)
subgroups_id

{'WT': ['1', '2', '3', '4', '5'],
 'zwf1^': ['1', '2', '3'],
 'pck1^': ['1', '2']}

time: 14.1 ms (started: 2023-06-04 19:02:19 -05:00)


### Save dataset and parameters

In [10]:
# save dataset
df_join_raw.to_csv("input/{}.csv".format(EXP), index=True)

# save parameters

# Data to be written
parameters = {
    "exp": exp,
    "groups_id": groups_id,
    "subgroups_id": subgroups_id
}
 
json_object = json.dumps(parameters)
with open("parameters.json", "w") as outfile:
    json.dump(parameters, outfile, indent=4)

time: 442 ms (started: 2023-06-04 19:02:19 -05:00)


### Create folders

In [11]:
# create experiments folder
try: 
    os.mkdir("output/{}".format(exp))
    os.mkdir("output/{}/preprocessing".format(exp))
    os.mkdir("output/{}/preprocessing/edges".format(exp))
    os.mkdir("output/{}/preprocessing/graphs".format(exp))
    os.mkdir("output/{}/preprocessing/graphs_data".format(exp))
    os.mkdir("output/{}/node_embeddings".format(exp))
    os.mkdir("output/{}/edge_embeddings".format(exp))
except OSError as error: 
    print(error)

time: 1.96 ms (started: 2023-06-04 19:02:19 -05:00)
